In [12]:
import nltk
import re
import string
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from collections import defaultdict
import random

In [2]:
def preprocessing(line, negative_contruct:list, smile_positive:list, smile_negative:list, ps:'Porter Stemmer', stop_words:list):
    line = re.sub(r'http\S+', '', line)  # remove URL
    line = re.sub('@[\w_]+', '', line)   # remove USER_MENTIONS
    line = re.sub('#[\w]+', '', line)   # remove hashtags
    line = line.split()
    for i, word in enumerate(line):     # replace negative constructs with not
        if word in negative_construct:
            line[i]="not"
        if word in smile_positive:    #replacing emojis with class name
            line[i] = "smile_positive"
        if word in smile_negative:
            line[i] = "smile_negative"
        line[i] = ps.stem(word)      # stemming
        
        if line[i] in stop_words:
            line[i] = ''
        line[i] = line[i].lower()
    line = ' '.join(line)

    translator = str.maketrans('', '', string.punctuation) # remove punctuations
    line = line.translate(translator)
    line = ' '.join(line.split())
    return line

In [3]:
file ='./Dataset.txt'
negative_construct = [ "can't", "wouldn't", "wasn't", "hadn't", "never", "won't"]

smile_pos = """:‑) :-] :-3 :-> 8-) :-} :o) :c) :^) =] =) :) :] :3 :> 8) :} :‑D :D 8‑D 8D x‑D xD
X‑D XD =D =3 B^D :-)) :'‑) :') :‑O :O :‑o :o :-0 8‑0 >:O :-* :* :× ;‑) ;) *-) *)
;‑] ;] ;^) :‑, ;D :‑P :P X‑P XP x‑p xp :‑p :p :‑Þ :‑Þ :‑þ :þ :Þ :Þ :‑b :b d: =p
>:P O:‑) O:) 0:‑3 0:3 0:‑) 0:) 0;^) |;‑) :‑J #‑) %‑) %) <3 @};- @}->--
@}‑;‑'‑‑‑ @>‑‑>‑‑""".split()

smile_neg = """:‑( :( :‑c :c :‑< :< :‑[ :[ :-|| >:[ :{ :@ >:( :'‑( :'( D‑': D:< D: D8 D; D= DX :‑/
:/ :‑. >:\ :L =L :S :‑| :| :‑X :X :‑# :# :‑& :& >:‑) >:) }:‑) }:) 3:‑) 3:) >;) ',:-l
',:-| >_> <_< <\3 </3
""".split()

ps = PorterStemmer()

stop_words = set(stopwords.words('english'))

In [4]:
lines = []
with open(file, encoding='utf-8') as f:
    for line in f:
        line = preprocessing(line, negative_construct, smile_pos, smile_neg, ps, stop_words)
        lines.append(line)

In [5]:
s = "I am the running won't .... :)"
print(preprocessing(s, negative_construct, smile_pos, smile_neg, ps, stop_words))

i run


In [9]:
print(lines[:10])

['1 da vinci code book awesome', '1 thi wa first clive cussler iv ever read even book like relic da vinci code plausibl this', '1 like da vinci code lot', '1 like da vinci code lot', '1 i like da vinci code ultimatli seem hold it own', '1 that even exagger midnight went walmart buy da vinci code amaz course', '1 i love da vinci code i want someth better different', '1 thought da vinci code wa great kite runner', '1 da vinci code actual good movie', '1 i thought da vinci code wa pretti good book']


In [13]:
def get_train_test(lines: 'list of sentences', percent):
    data = []
    for line in lines:
        outcome = int(line.split()[0])
        sentence = ' '.join(line.split()[1:])
        temp = [sentence, outcome]
        data.append(temp)
    
    random.shuffle(data)
    split_index = int(percent*len(data))
    train_data = data[:split_index]
    test_data = data[split_index+1:]
    
    return(train_data, test_data)        

In [14]:
train_data, test_data = get_train_test(lines, 0.75)

In [17]:
print(len(train_data), len(test_data))


5314 1771


In [33]:
def generate_probabilities(dataset:'list[0] = sentence, list[1]= outcome'):
    prob = defaultdict(int)
    outcome_set = set()
    # first store the counts of (word, outcome)
    for sentence, outcome in dataset:
        outcome_set.add(outcome)
        prob[(outcome, None)]+=1 # (outcome, None) gives count of the outcome
        for word in sentence.split():
            tup = (word, outcome)
            prob[tup]+=1
            
    # convert the counts to probabilities
    for tup, count in prob.items():
        if tup[1] is not None:
            prob[tup] = prob[tup]/prob[(tup[1], None)] # tup[1] is the outcome and (outcome, None) has count of the outcome
    # convert count of outcome to prob
    for outcome in outcome_set:
        prob[(outcome, None)] = prob[(outcome, None)] / len(dataset)
    return (prob, outcome_set)

In [34]:
probabilities, possible_outcomes = generate_probabilities(train_data)

In [35]:
print(probabilities[(1,None)])
print(probabilities[(0,None)])
print(probabilities)

0.5639819345126083
0.4360180654873918
defaultdict(<class 'int'>, {(1, None): 0.5639819345126083, ('harri', 1): 0.2682682682682683, ('potter', 1): 0.26860193526860193, ('awesom', 1): 0.12078745412078745, ('i', 1): 0.6196196196196196, ('care', 1): 0.02135468802135469, ('anyon', 1): 0.02135468802135469, ('say', 1): 0.023690357023690357, ('differently', 1): 0.021021021021021023, ('mission', 1): 0.25225225225225223, ('imposs', 1): 0.20954287620954287, ('3', 1): 0.06106106106106106, ('wa', 1): 0.21154487821154488, ('excellent', 1): 0.021688355021688355, (0, None): 0.4360180654873918, ('combin', 0): 0.028485110056107037, ('opinion', 0): 0.0293482952093224, ('review', 0): 0.029779887785930083, ('gari', 0): 0.028485110056107037, ('gin', 0): 0.028485110056107037, ('zen', 0): 0.028485110056107037, ('da', 0): 0.3336210617177385, ('vinci', 0): 0.3336210617177385, ('code', 0): 0.3331894691411308, ('sucks', 0): 0.11221406991799741, ('mission', 0): 0.028485110056107037, ('imposs', 0): 0.02762192490289

In [40]:
def bayes(probabilities:'tuple to prob', sentence, possible_outcomes):
    best_outcome = -1
    best_prob = -1
    for outcome in possible_outcomes:
        cur_prob = 1
        for word in sentence.split():
            tup = (word, outcome)
            cur_prob*=probabilities[tup]
        if cur_prob > best_prob:
            best_prob = cur_prob
            best_outcome = outcome
    return best_outcome
    

In [41]:
correct_and_predicted = [] # list which stores correct outcome and predicted outcome for accuracy, f1 etc. temp[0] is correct. temp[1] is predicted
for sentence, outcome in test_data:
    predict = bayes(probabilities, sentence, possible_outcomes)
    temp = [outcome, predict]
    correct_and_predicted.append(temp)

[1, 1]
[0, 0]
[0, 1]
[1, 1]
[0, 0]
[1, 0]
[1, 1]
[0, 0]
[0, 0]
[0, 0]
[1, 1]
[1, 1]
[0, 0]
[0, 0]
[0, 0]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[0, 0]
[0, 0]
[1, 1]
[0, 0]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[1, 1]
[0, 0]
[0, 0]
[1, 1]
[1, 1]
[0, 0]
[1, 1]
[1, 1]
[1, 1]
[0, 0]
[1, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[1, 1]
[0, 0]
[1, 1]
[1, 1]
[1, 1]
[0, 0]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[0, 0]
[0, 0]
[1, 1]
[0, 0]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[1, 0]
[0, 0]
[0, 0]
[1, 1]
[0, 0]
[0, 0]
[1, 1]
[0, 0]
[1, 1]
[0, 0]
[0, 0]
[1, 1]
[1, 0]
[0, 0]
[0, 0]
[0, 0]
[1, 1]
[0, 0]
[0, 0]
[0, 0]
[1, 1]
[1, 1]
[0, 1]
[0, 0]
[0, 0]
[1, 1]
[0, 0]
[0, 0]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[0, 0]
[0, 0]
[0, 0]
[1, 1]
[1, 1]
[1, 1]
[1, 1]
[0, 0]
[0, 0]
[1, 1]
[0, 0]
[1, 1]
[1, 1]
[0, 0]
[1, 1]
[1, 1]
[0, 0]
[1, 1]
[1, 1]
[0, 0]
[0, 0]
[0, 0]
[1, 1]
[0, 0]
[1, 1]
[1, 1]
[0, 0]
[1, 0]
[0, 0]
[1, 1]
[0, 0]
[1, 1]
[0, 0]
[0, 0]
[1, 1]
[0, 0]
[1, 1]
[1, 1]